In [1]:
from random import uniform
import time
from IPython.display import display, clear_output
import numpy as np

In [2]:
# Funktsioon, mida saaks pärast ära kasutada selleks, et joonistada olemasoleva laua üle, mitte pidevalt uuele reale
def black_box(): 
  i = 1
  while True:
    clear_output(wait=True)
    display('Iteration '+str(i)+' Score: '+str(uniform(0, 1)))
    time.sleep(0.2)
    i += 1

In [3]:
# Mitu nuppu kummalgi mängijal laua läbinud on
PLAYER1_COMPLETED = 0 
PLAYER2_COMPLETED = 0

# Mis positsioonidel nupud mängus on
PLAYER1_POS_IN_PLAY = set() 
PLAYER2_POS_IN_PLAY = set()

# Defineerib võimalikud asukohad mõlema mängija jaoks
PLAYER1_TILES = {1 : [0,3], 2 : [0,2], 3 : [0,1], 4 : [0,0], 5 : [1,0], 6 : [1,1], 7 : [1,2], 8 : [1,3], 9 : [1,4], 10 : [1,5], 11 : [1,6], 12 : [1,7], 13 : [0,7], 14 : [0,6]} 
PLAYER2_TILES = {1 : [2,3], 2 : [2,2], 3 : [2,1], 4 : [2,0], 5 : [1,0], 6 : [1,1], 7 : [1,2], 8 : [1,3], 9 : [1,4], 10 : [1,5], 11 : [1,6], 12 : [1,7], 13 : [2,7], 14 : [2,6]} 

# Mis positsioonidel kumbki mängija sama käigu jooksul uuesti veeretada saab
ROLL_AGAIN_TILES = [4, 8, 14]

In [4]:
def increaseScore(player):
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED
    if (player == 1):
        PLAYER1_COMPLETED += 1
    else:
        PLAYER2_COMPLETED += 1
    return

In [5]:
def getScore(player):
    if (player == 1):
        return PLAYER1_COMPLETED
    else:
        return PLAYER2_COMPLETED

In [6]:
def resetGameState():
    global PLAYER1_COMPLETED, PLAYER2_COMPLETED, PLAYER1_POS_IN_PLAY, PLAYER2_POS_IN_PLAY

    PLAYER1_COMPLETED = 0 
    PLAYER2_COMPLETED = 0

    PLAYER1_POS_IN_PLAY = set() 
    PLAYER2_POS_IN_PLAY = set()

In [7]:
# Tagastab:
# 0 0 0 0     0 0
# 0 0 0 0 0 0 0 0
# 0 0 0 0     0 0
def initBoard():
    resetGameState()
    board = np.zeros(shape=(3,8)).astype(int).astype(str)
    board[0][4:6] = " "
    board[2][4:6] = " "
    return board

In [8]:
def prettyPrint(board):
    print("Skoor " + str(PLAYER1_COMPLETED) + " : " + str(PLAYER2_COMPLETED) + "\n")
    print('\n'.join([' '.join([str(cell) for cell in row]) for row in board]) + "\n")
prettyPrint(initBoard())

Skoor 0 : 0

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0



In [9]:
def rollDice():
    result = 0
    for i in range(4):
        result += np.random.randint(2)
    return result

In [10]:
# Lisab antud mängija nupu antud positsioonile laual ja tagastab laua
def addPiece(board, player, addPieceLocIdx):
    if addPieceLocIdx < 1 or addPieceLocIdx > 4:
        print("Can't add piece on this index, as the range of options is 1-4.")
        return board, False

    playerMovePath = PLAYER1_TILES     if player == 1 else PLAYER2_TILES
    piecesInPlay = PLAYER1_POS_IN_PLAY if player == 1 else PLAYER2_POS_IN_PLAY  

    if addPieceLocIdx in piecesInPlay:
        print("Given position already contains a piece.")
        return board, False

    x, y = playerMovePath.get(addPieceLocIdx)
    piecesInPlay.add(addPieceLocIdx)
    board[x][y] = str(player)
    return board, True

board = initBoard()
addPiece(board, 2, 2)
addPiece(board, 1, 2)
prettyPrint(board)

Skoor 0 : 0

0 0 1 0     0 0
0 0 0 0 0 0 0 0
0 0 2 0     0 0



In [11]:
def movePiece(board, player, currentPosLocIdx, steps):
    playerMovePath = PLAYER1_TILES     if player == 1 else PLAYER2_TILES
    piecesInPlay = PLAYER1_POS_IN_PLAY if player == 1 else PLAYER2_POS_IN_PLAY
    
    enemyPiecesInPlay = PLAYER2_POS_IN_PLAY if player == 1 else PLAYER1_POS_IN_PLAY  

    if steps < 1:
        return board, False
    
    if steps > 4:
        print("Not a valid dice roll result")
        return board, False

    if currentPosLocIdx not in piecesInPlay:
        print("There is no gamepiece on the given tile. For player " + str(player) + ", there are pieces in play on the following positions: " + str(piecesInPlay))
        prettyPrint(board)
        return board, False

    if (currentPosLocIdx + steps) in piecesInPlay:
        print("There already is a gamepiece on the target tile. You can't step on your own piece.")
        prettyPrint(board)
        return board, False
    
    old_x, old_y = playerMovePath.get(currentPosLocIdx)

    if (currentPosLocIdx + steps) > 14:
        board[old_x][old_y] = str(0)
        piecesInPlay.remove(currentPosLocIdx)
        increaseScore(player)

        if (getScore(player) >= 7):
            print("Mängija " + str(player) + " võitis!")

        return board, True

    new_x, new_y = playerMovePath.get(currentPosLocIdx + steps)

    board[old_x][old_y] = str(0)
    board[new_x][new_y] = str(player)
    piecesInPlay.remove(currentPosLocIdx)
    piecesInPlay.add(currentPosLocIdx + steps)

    if (currentPosLocIdx + steps) in enemyPiecesInPlay and (currentPosLocIdx + steps) > 4 and (currentPosLocIdx + steps) < 13:
        enemyPiecesInPlay.remove(currentPosLocIdx + steps)

    return board, True

In [12]:
board = initBoard()
for i in range(7):
    addPiece(board, 2, 2)
    movePiece(board, 2, 2, 4)
    movePiece(board, 2, 6, 4)
    movePiece(board, 2, 10, 4)
    movePiece(board, 2, 14, 4)
    prettyPrint(board)

Skoor 0 : 1

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Skoor 0 : 2

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Skoor 0 : 3

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Skoor 0 : 4

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Skoor 0 : 5

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Skoor 0 : 6

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0

Mängija 2 võitis!
Skoor 0 : 7

0 0 0 0     0 0
0 0 0 0 0 0 0 0
0 0 0 0     0 0



In [13]:
def play():
    player = 1
    board = initBoard()
    while getScore(1) < 7 or getScore(2) < 7:
        clear_output(wait=True)
        print(PLAYER1_POS_IN_PLAY)
        print(PLAYER2_POS_IN_PLAY)
        diceResult = rollDice()
        print("----- Turn start for player " + str(player) + " ----- \n")
        print("Dice roll result: " + str(diceResult))

        prettyPrint(board)

        if (diceResult == 0):
            print("Skipped turn due to 0 dice roll")
            print("----- Turn over for player " + str(player) + " ----- \n")
            player = 3 - player
            continue

        turnSuccessful = False
        while (turnSuccessful != True):
            try:
                action = -1
                while (action != "M" and action != "P"): 
                    action = input("Do you want to place a new piece or move an existing piece (P to place, M to move)? ")[0]
                
                    if (action == "P"):
                        turnSuccessful = addPiece(board, player, diceResult)[1]

                    if (action == "M"):
                        currentPosLocIdx = -1
                        while not (currentPosLocIdx > 0 and currentPosLocIdx < 15): 
                            currentPosLocIdx = int(input("Enter position of piece you'd like to move "))
                            print(currentPosLocIdx)
            
                        turnSuccessful = movePiece(board, player, currentPosLocIdx, diceResult)[1]
            except:
                print("Invalid input")
                continue

        print("----- Turn over for player " + str(player) + " ----- \n")

        player = 3 - player
        

In [ ]:
play()

{8, 2}
{1, 4}
----- Turn start for player 2 ----- 

Dice roll result: 3
Skoor 0 : 0

0 0 1 0     0 0
0 0 0 1 0 0 0 0
2 0 0 2     0 0

Invalid input
